# Scraping French News

## Helper functions

### clean_file : Save older scaping result

In [1]:
# save before scraping

import shutil
# importing os module
import os
# import datetime module
import datetime


def clean_file(path_file_name):
    '''
    Clean file already traited : rename file with date
    '''
    try:
        d = datetime.datetime.now()
        str_date = '_' + d.strftime("%Y%m%d_%H_%M_%S")
        res_re = re.search('\.\w+$',path_file_name)
        path_file_name_saved = \
            path_file_name[0:res_re.start()] + str_date + res_re.group(0)
            
        shutil.move(path_file_name, path_file_name_saved) 
        print('File {} moved!'.format(path_file_name_saved))
    except:
        print('File {} does not exist!'.format(path_file_name))
        


### run_spider : running spider several times 

In [2]:
import scrapy
import scrapy.crawler as crawler
from multiprocessing import Process, Queue
from twisted.internet import reactor

# the wrapper to make it run more times
def run_spider(spider):
    '''
    function to run several times scraping process
    '''
    def f(q):
        try:
            runner = crawler.CrawlerRunner()
            deferred = runner.crawl(spider)
            deferred.addBoth(lambda _: reactor.stop())
            reactor.run()
            q.put(None)
        except Exception as e:
            q.put(e)

    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()

    if result is not None:
        raise result

## Scraping Legorafi

### Class definitions

In [3]:
import scrapy
from scrapy.loader import ItemLoader
from scrapy.loader.processors import MapCompose, Join, TakeFirst
from w3lib.html import remove_tags

class GorafiItem(scrapy.Item):
    '''
    Class item to declare different information to scrap
    and how to process (as input or output)
    '''
    # define the fields for your item here like:
    url = scrapy.Field(output_processor=TakeFirst())
    
    source = scrapy.Field(output_processor=TakeFirst())
    
    author = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    title = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    theme = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    date_published = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    description = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    body = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=Join()
    )

In [4]:
class GorafiSpider(scrapy.Spider):
    '''
    Spider to scrap over Le Gorafi webpages : 
    - how to find information for scraping
    - which field names to store
    '''
    # Your spider definition
    name = 'news_gorafi_spider'
    # output definition :
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'items_gorafi.json'
  }
    # urls to scrap
    start_urls = [
        'http://www.legorafi.fr/2019/12/17/psycho-comment-guerir-dun-chagrin-damour-en-donnant-tout-son-argent-au-gorafi/'
    ]

    def parse(self, response):
        '''
        Parse definition with xpath which define all patterns to use
        for retrieve information into HTML strings
        '''
        #url	source	author	title	theme	description	date_published	body   
        l = ItemLoader(item=GorafiItem(), selector=response)
        
        l.add_value('url', response.url)
        
        l.add_value('source', "LeGorafi")
        
        l.add_xpath('author', 
            "//section[@class='metas container']/span[@class='context']/a")
        
        l.add_xpath("title", "//h1")
        
        l.add_xpath("theme", "//a[@rel='category tag']")
        
        l.add_xpath("description", ".//div[@class='intro']/p")
        
        l.add_xpath("date_published", 
            "//section[@class='metas container']/span[@class='context']",
            re="[0-9]+/[0-9]+/[0-9]+")
       
        l.add_xpath("body", "//div[@class='content']/p")
        
        yield l.load_item()

In [5]:
import scrapy

class GorafiRssSpider(scrapy.Spider):
    '''
    Class Spider for retrieving all links to news webpages from Le Gorafi RSS
    '''
    name = "gorafi_rss"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'gorafi_rss.json'
    }
    
    def start_requests(self):
        urls = [
                'http://www.legorafi.fr/feed/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
     
    def parse(self, response):
        for post in response.xpath('//channel/item'):
            yield {
                'title' : post.xpath('title//text()').extract_first(),
                'link': post.xpath('link//text()').extract_first(),
                'pubDate' : post.xpath('pubDate//text()').extract_first(),
            }


In [6]:
import scrapy
import re

class GorafiPageSpider(scrapy.Spider):
    '''
    Spider to scrap all Le Gorafi pages from selected category
    Configure : 
    - GorafiPageSpider.custom_settings : save location 
    - num_max_pages : the number of next page to scrap
    - url_first_page : web page to start with
    '''
    name = "gorafi_page"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'pages_gorafi.json'
    }
    
    num_max_pages = 1
    
    url_first_page = 'http://www.legorafi.fr/category/france/societe/'
    
    def start_requests(self):
        urls = [
                self.url_first_page,
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
    
    def parse(self, response):
        for post in response.xpath('//article/h2'):
            yield {
                'link': post.xpath('a/@href').extract_first()
            }
            
        next_page = response.xpath(
            '//a[@class="next page-numbers"]/@href').get()
        
        if next_page is not None:
            try:
                num_next_page = int(re.search("(?<=/)\d+(?=/$)", 
                         next_page).group(0))
                
                if (num_next_page < self.num_max_pages):
                    #next_page = response.urljoin(next_page)
                    yield scrapy.Request(next_page, callback=self.parse)
            except:
                next_page = None
                

### Scraping RSS urls

#### Definitions

In [8]:
# path to results file for urls
PATH_RSS_PAGES_GORAFI = '../../data/gorafi_rss_urls.json'
# declare your data location for scraping rss links 
PATH_RSS_NEWS_GORAFI = '../../data/gorafi_rss_pages.json'

#### Scraping urls links

In [9]:
# clean (move file if exist)
clean_file(PATH_RSS_PAGES_GORAFI)

File ../../data/gorafi_rss_urls.json does not exist!


In [10]:
# Configure spider
GorafiRssSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_RSS_PAGES_GORAFI
}
# scraping
run_spider(GorafiRssSpider)

In [11]:
import pandas as pd

df_rss_gorafi = pd.read_json(PATH_RSS_PAGES_GORAFI)
df_rss_gorafi

,link,pubDate,title
0,http://www.legorafi.fr/2019/12/20/le-pere-noel...,"Fri, 20 Dec 2019 13:00:49 +0000",Le Père Noël découvre stupéfait l’existence de...
1,http://www.legorafi.fr/2019/12/20/gorafi-magaz...,"Fri, 20 Dec 2019 09:00:04 +0000",Gorafi Magazine : Partir à la retraite à 115 ans
2,http://www.legorafi.fr/2019/12/19/noel-pour-co...,"Thu, 19 Dec 2019 13:00:32 +0000","Noël – Pour coller à la réalité, les magasins ..."
3,http://www.legorafi.fr/2019/12/19/le-gouvernem...,"Thu, 19 Dec 2019 09:22:54 +0000",Le gouvernement autorise les chauffeurs des bu...
4,http://www.legorafi.fr/2019/12/18/espagne-18-m...,"Wed, 18 Dec 2019 09:14:41 +0000",Espagne – 18 morts dans la traditionnelle bata...
5,http://www.legorafi.fr/2019/12/17/psycho-comme...,"Tue, 17 Dec 2019 09:03:56 +0000",Psycho : Comment guérir d’un chagrin d’amour e...
6,http://www.legorafi.fr/2019/12/16/apres-sa-dem...,"Mon, 16 Dec 2019 13:00:39 +0000",Après sa démission Jean-Paul Delevoye savoure ...
7,http://www.legorafi.fr/2019/12/16/le-tfc-va-sa...,"Mon, 16 Dec 2019 09:04:56 +0000",Le TFC va sacrifier un de ses joueurs pour cal...
8,http://www.legorafi.fr/2019/12/16/horoscope-du...,"Mon, 16 Dec 2019 08:31:26 +0000",Horoscope du 16 décembre 2019
9,http://www.legorafi.fr/2019/12/13/les-climato-...,"Fri, 13 Dec 2019 14:30:12 +0000",Les climato-sceptiques présentent leur Greta ...


In [12]:
df_rss_gorafi.shape

(20, 3)

#### Scraping RSS news

In [13]:
# configure : add all retrieved links to Spider
GorafiSpider.start_urls = df_rss_gorafi["link"].tolist()
GorafiSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_RSS_NEWS_GORAFI
}

# scraping
run_spider(GorafiSpider)

In [14]:
import pandas as pd

df_gorafi = pd.read_json(PATH_RSS_NEWS_GORAFI)
df_gorafi.head()

,author,body,date_published,description,source,theme,title,url
0,La Rédaction,« Je n’en revenais pas… C’est un lutin stagiai...,20/12/2019,A quatre jours de la distribution des cadeaux ...,LeGorafi,Société,Le Père Noël découvre stupéfait l’existence de...,http://www.legorafi.fr/2019/12/20/le-pere-noel...
1,La Rédaction,NaN,20/12/2019,NaN,LeGorafi,Magazine,Gorafi Magazine : Partir à la retraite à 115 ans,http://www.legorafi.fr/2019/12/20/gorafi-magaz...
2,La Rédaction,« Mes équipes sentaient le malaise s’installer...,19/12/2019,"Afin d’être plus en phase avec l’actualité, le...",LeGorafi,Société,"Noël – Pour coller à la réalité, les magasins ...",http://www.legorafi.fr/2019/12/19/noel-pour-co...
3,La Rédaction,Ainsi les chauffeurs sont autorisés à prendre ...,19/12/2019,Paris – Pour aider les Français à pouvoir prof...,LeGorafi,Société,Le gouvernement autorise les chauffeurs des bu...,http://www.legorafi.fr/2019/12/19/le-gouvernem...
4,La Rédaction,Oliver Sadran espère ainsi conjurer le mauvais...,16/12/2019,Aux grands maux les grands remèdes. Afin de ca...,LeGorafi,Sports,Le TFC va sacrifier un de ses joueurs pour cal...,http://www.legorafi.fr/2019/12/16/le-tfc-va-sa...


In [15]:
df_gorafi.shape

(20, 8)

### Scraping urls into webpages 

#### Definitions

In [165]:
URL_PAGES_GORAFI_SOCIETE = 'http://www.legorafi.fr/category/france/societe/'
# declare your data location
PATH_PAGES_GORAFI_SOCIETE = '../../data/pages_gorafi.json'
PATH_NEWS_GORAFI_SOCIETE = '../../data/gorafi_societe.json'

URL_PAGES_GORAFI_POLITIQUE = 'http://www.legorafi.fr/category/france/politique/'
# declare your data location
PATH_PAGES_GORAFI_POLITIQUE = '../../data/pages_gorafi_politique.json'
PATH_NEWS_GORAFI_POLITIQUE = '../../data/gorafi_politique.json'

#### Le Gorafi Société

##### Scraping links

In [153]:
# clean (move file if exist)
clean_file(PATH_PAGES_GORAFI_SOCIETE)

File ../../data/pages_gorafi_20191225_10_47_30.json moved!


In [104]:
# configure scraping
GorafiPageSpider.url_first_page = URL_PAGES_GORAFI_SOCIETE
GorafiPageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_PAGES_GORAFI_SOCIETE
    }
GorafiPageSpider.num_max_pages = 50

# clean last output
clean_file(PATH_PAGES_GORAFI_SOCIETE)
# scraping page urls LeGorafi
run_spider(GorafiPageSpider)

In [161]:
import pandas as pd

df_gorafi_pages = pd.read_json(PATH_PAGES_GORAFI_SOCIETE)
df_gorafi_pages.head()

,link
0,http://www.legorafi.fr/2019/12/19/noel-pour-co...
1,http://www.legorafi.fr/2019/12/19/le-gouvernem...
2,http://www.legorafi.fr/2019/12/17/psycho-comme...
3,http://www.legorafi.fr/2019/12/12/test-quel-es...
4,http://www.legorafi.fr/2019/12/11/plusieurs-bu...


In [166]:
df_gorafi_pages.shape

(246, 1)

##### Scraping news

In [167]:
# configure : add all retrieved links to Spider
GorafiSpider.start_urls = df_gorafi_pages["link"].tolist()
GorafiSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_NEWS_GORAFI
    }
# scraping
run_spider(GorafiSpider)

In [168]:
import pandas as pd

df_gorafi_soc = pd.read_json(PATH_NEWS_GORAFI)
df_gorafi_soc.head()

,author,body,date_published,description,source,theme,title,url
0,La Rédaction,« Mes équipes sentaient le malaise s’installer...,19/12/2019,"Afin d’être plus en phase avec l’actualité, le...",LeGorafi,Société,"Noël – Pour coller à la réalité, les magasins ...",http://www.legorafi.fr/2019/12/19/noel-pour-co...
1,La Rédaction,"Oui, car Mathias a économisé pas moins de 5 eu...",10/12/2019,"Alors qu’il rentrait d’un week-end, Mathias a ...",LeGorafi,Société,Il économise 5 euros en achetant au duty-free ...,http://www.legorafi.fr/2019/12/10/il-economise...
2,La Rédaction,1/ Faire un beau chèque en s’appliquant pour l...,17/12/2019,L’échec amoureux n’a plus de secret pour vous ...,LeGorafi,Société,Psycho : Comment guérir d’un chagrin d’amour e...,http://www.legorafi.fr/2019/12/17/psycho-comme...
3,La Rédaction,\nwindow.beOpAsyncInit = function() {\n BeOpS...,12/12/2019,"Depuis les annonces d’Edouard Philippe, vous v...",LeGorafi,Société,Test : quel est l’âge idéal pour mourir avant ...,http://www.legorafi.fr/2019/12/12/test-quel-es...
4,La Rédaction,"Selon les services sanitaires, se sont plusieu...",11/12/2019,Paris – La chaîne de restauration rapide lancé...,LeGorafi,Société,Plusieurs Burger Quiz fermés après une visite ...,http://www.legorafi.fr/2019/12/11/plusieurs-bu...


In [169]:
df_gorafi_soc.shape

(246, 8)

#### Le Gorafi Politique

##### Scraping links

In [155]:
# configure scraping
GorafiPageSpider.url_first_page = URL_PAGES_GORAFI_POLITIQUE
GorafiPageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_PAGES_GORAFI_POLITIQUE
    }
GorafiPageSpider.num_max_pages = 50

# move file if exist
clean_file(PATH_PAGES_GORAFI_POLITIQUE)

# scraping pages LeGorafi
run_spider(GorafiPageSpider)

File ../../data/pages_gorafi_politique.json does not exist!


In [163]:
import pandas as pd

df_gorafi_pages_pol = pd.read_json(PATH_PAGES_GORAFI_POLITIQUE)
df_gorafi_pages_pol.head()

,link
0,http://www.legorafi.fr/2019/12/11/jean-paul-de...
1,http://www.legorafi.fr/2019/12/09/jean-paul-de...
2,http://www.legorafi.fr/2019/12/05/christophe-c...
3,http://www.legorafi.fr/2019/11/25/lrem-edouard...
4,http://www.legorafi.fr/2019/11/22/inquiete-par...


In [170]:
df_gorafi_pages_pol.shape

(246, 1)

##### Scraping news

In [171]:
# configure : add all retrieved links to Spider
GorafiSpider.start_urls = df_gorafi_pages_pol["link"].tolist()
GorafiSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_NEWS_GORAFI_POLITIQUE
    }
# scraping
run_spider(GorafiSpider)

In [172]:
import pandas as pd

df_gorafi_pol = pd.read_json(PATH_NEWS_GORAFI_POLITIQUE)
df_gorafi_pol.head()

,author,body,date_published,description,source,theme,title,url
0,La Rédaction,« C’est juste un petit cadeau pour le service ...,09/12/2019,Paris – Nouvelle polémique pour Jean-Paul Dele...,LeGorafi,Politique,Jean-Paul Delevoye a oublié de préciser qu’il ...,http://www.legorafi.fr/2019/12/09/jean-paul-de...
1,La Rédaction,De nombreuses sources citent Marlène Schiappa ...,25/11/2019,"De nombreux détails croustillants, concernant ...",LeGorafi,Politique,LREM : Édouard Philippe rebaptisé Eduardo Fili...,http://www.legorafi.fr/2019/11/25/lrem-edouard...
2,La Rédaction,« Oui c’est un peu regrettable mais j’ai quitt...,11/12/2019,"Paris – Face aux polémiques naissantes, Jean-P...",LeGorafi,Politique,Jean-Paul Delevoye contraint de démissionner d...,http://www.legorafi.fr/2019/12/11/jean-paul-de...
3,La Rédaction,« La précarité qu’ils traversent leur sera de ...,13/11/2019,Alors que la mobilisation étudiante prend de l...,LeGorafi,Politique,Emmanuel Macron aux étudiants « Je n’adore pas...,http://www.legorafi.fr/2019/11/13/emmanuel-mac...
4,La Rédaction,Alors que de nombreux manifestants sont attend...,05/12/2019,Après que l’Etat a commandé de nouveaux LBD à ...,LeGorafi,Politique,Christophe Castaner promet que les LBD « feron...,http://www.legorafi.fr/2019/12/05/christophe-c...


In [173]:
df_gorafi_pol.shape

(246, 8)